### Проверка ДЗ 1

In [90]:
import json

In [91]:
# Указываем номер варианта проверяемого
variant = 16

# Считываем исходные данные
with open(f"homework_1/input_data/{variant}.json") as f:
    input_data = json.load(f)

In [92]:
# Предварительно лучше перейти в ветку которая на PR, чтобы файл с ответом уже был в папке и его не скачивать отдельно
# Как это сделать: https://codex.so/git-checkout-on-pull-request-from-fork

# Путь к файлу с ответом ученика
output_path = "homework_1/muravtsev/output.json"

# Считываем выходные данные
with open(output_path) as f:
    output_data = json.load(f)

In [93]:
input_data

{'gamma_water': 1.0531826688400516,
 'md_vdp': 2795.4461926851586,
 'd_tub': 0.6292958170855484,
 'angle': 48.26556535739659,
 'roughness': 0.00023828101188969189,
 'p_wh': 151.89057740712013,
 't_wh': 26.056301710698456,
 'temp_grad': 2.211303276181458}

In [94]:
from math import sin, pi

In [95]:
input_data["md_vdp"]*sin(input_data["angle"]*pi/180)

2086.0688620165038

In [96]:
# Получаем эталонный ответ с помощью uniflocpy
import pandas as pd
import unifloc.tools.units_converter as uc
import unifloc.common.trajectory as traj
import unifloc.common.ambient_temperature_distribution as amb
import unifloc.pipe.pipeline as pipel
import unifloc.pvt.fluid_flow as fl

# Инициализация исходных данных класса Pipeline
trajectory = traj.Trajectory(pd.DataFrame(columns=["MD", "TVD"],
                                     data=[[0, 0],
                                           [input_data["md_vdp"], input_data["md_vdp"]*sin(input_data["angle"]*pi/180)]]))

ambient_temperature_data = {"MD": [0, input_data["md_vdp"]], "T": [input_data["t_wh"]+273.15, 
                                                                   input_data["t_wh"]+273.15 + 2.71 / 100 * input_data["md_vdp"]*sin(input_data["angle"]*pi/180)]}

amb_temp = amb.AmbientTemperatureDistribution(ambient_temperature_data)

fluid_data = {
    "q_fluid": uc.convert_rate(100, "m3/day", "m3/s"),
    "pvt_model_data": {
        "black_oil": {
            "gamma_gas": 0.7, "gamma_wat": input_data["gamma_water"], "gamma_oil": 0.8,
            "wct": 1, "phase_ratio": {"type": "GOR", "value": 401}}}}

fluid = fl.FluidFlow(**fluid_data)

d = input_data["d_tub"] / 10
top_depth = 0
bottom_depth = input_data["md_vdp"]
roughness = input_data["roughness"]
p_wh = input_data["p_wh"] * 101325  # Па
flow_direction = -1
# Инициализация объекта pvt-модели
pipe = pipel.Pipeline(top_depth, bottom_depth, d, roughness,
                      trajectory, fluid, amb_temp)

# Расчёт VLP
q_liq_array = output_data["q_liq"]
p_wf_array = []
for q_liq in q_liq_array:
    p_wf, *_ = pipe.calc_pt("top", p_wh, flow_direction, q_liq/86400, 1)
    p_wf_array.append(p_wf/101325)

In [98]:
# Сравним 2 массива
import numpy.testing

# Допустимый процент ошибки
rtol = 0.02
try:
    numpy.testing.assert_allclose(p_wf_array, output_data["p_wf"], rtol=rtol)
except AssertionError as e:
    
    print("Ответ ученика неверный")
    
    print(e)